# Machine Learning on Collisions

This notebook will look at exploring the potential use in ML for predicting an r value based on the intersection. The uses of this can be for design or redesign of an intersection. There are two sets of features that will be used for this modelling, both indicated as a chapter. The code will do the following for each chapter:

1. Import and feature engineer the master_gdf as well as import intersection_r.
2. Combine intersection_r and master_gdf based on intersection.
3. Explore # of features to optimize ML score.
4. Explore potential hyperparameters to optimize score.
5. Test the ML model and assess how viable it is in finding r.

For this analysis, we will be using XGBoost Regressor to find r.

## 1.1.1 Importing Packages and Functions for feature engineering

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

from sklearn.metrics import mean_absolute_error as mae

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import xgboost as xgb
from xgboost import XGBRegressor

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

In [2]:
def feature_engineering(data):
    
    #ONE
    # Calculate the count of occurrences of each neighborhood as a proxy for collisions
    collisions_per_neighborhood = data['NEIGHBOURHOOD_158'].value_counts().reset_index()
    collisions_per_neighborhood.columns = ['NEIGHBOURHOOD_158', 'collisions_count']

    # Calculate total collisions for all neighborhoods
    total_collisions_all = collisions_per_neighborhood['collisions_count'].sum()

    # Compute ratio of collisions per neighborhood to total collisions of all neighborhoods
    collisions_per_neighborhood['collision_ratio_neighbourhood'] = collisions_per_neighborhood['collisions_count'] / total_collisions_all

    # Merge collision ratio information back into master_gdf
    data = data.merge(collisions_per_neighborhood[['NEIGHBOURHOOD_158', 'collision_ratio_neighbourhood']], on='NEIGHBOURHOOD_158', how='left')
    
    
    #TWO
    # Calculate the count of occurrences of each neighborhood as a proxy for collisions
    collisions_per_ward = data['WARDNUM'].value_counts().reset_index()
    collisions_per_ward.columns = ['WARDNUM', 'collisions_count']

    # Calculate total collisions for all neighborhoods
    total_collisions_wards = collisions_per_ward['collisions_count'].sum()

    # Compute ratio of collisions per neighborhood to total collisions of all neighborhoods
    collisions_per_ward['collision_ratio_ward'] = collisions_per_ward['collisions_count'] / total_collisions_wards

    # Merge collision ratio information back into master_gdf
    data = data.merge(collisions_per_ward[['WARDNUM', 'collision_ratio_ward']], on='WARDNUM', how='left')
    
    
    #THREE
    #this one uses traffic volumes as the differentiator between neighbourhoods, we can also change this so that it fits wards/districts
    daily_traffic = [
    'daily_sb_cars_r', 'daily_sb_cars_t', 'daily_sb_cars_l', 
    'daily_nb_cars_r', 'daily_nb_cars_t', 'daily_nb_cars_l', 
    'daily_wb_cars_r', 'daily_wb_cars_t', 'daily_wb_cars_l', 
    'daily_eb_cars_r', 'daily_eb_cars_t', 'daily_eb_cars_l', 
    'daily_sb_truck_r', 'daily_sb_truck_t', 'daily_sb_truck_l', 
    'daily_nb_truck_r', 'daily_nb_truck_t', 'daily_nb_truck_l', 
    'daily_wb_truck_r', 'daily_wb_truck_t', 'daily_wb_truck_l', 
    'daily_eb_truck_r', 'daily_eb_truck_t', 'daily_eb_truck_l', 
    'daily_sb_bus_r', 'daily_sb_bus_t', 'daily_sb_bus_l', 
    'daily_nb_bus_r', 'daily_nb_bus_t', 'daily_nb_bus_l', 
    'daily_wb_bus_r', 'daily_wb_bus_t', 'daily_wb_bus_l', 
    'daily_eb_bus_r', 'daily_eb_bus_t', 'daily_eb_bus_l', 
    'daily_nx_peds', 'daily_sx_peds', 'daily_ex_peds', 'daily_wx_peds', 
    'daily_nx_bike', 'daily_sx_bike', 'daily_ex_bike', 'daily_wx_bike', 
    'daily_nx_other', 'daily_sx_other', 'daily_ex_other', 'daily_wx_other'
    ]

    hourly_traffic = [ 'hourly_sb_cars_r', 'hourly_sb_cars_t', 'hourly_sb_cars_l',
    'hourly_nb_cars_r', 'hourly_nb_cars_t', 'hourly_nb_cars_l',
    'hourly_wb_cars_r', 'hourly_wb_cars_t', 'hourly_wb_cars_l',
    'hourly_eb_cars_r', 'hourly_eb_cars_t', 'hourly_eb_cars_l',
    'hourly_sb_truck_r', 'hourly_sb_truck_t', 'hourly_sb_truck_l',
    'hourly_nb_truck_r', 'hourly_nb_truck_t', 'hourly_nb_truck_l',
    'hourly_wb_truck_r', 'hourly_wb_truck_t', 'hourly_wb_truck_l',
    'hourly_eb_truck_r', 'hourly_eb_truck_t', 'hourly_eb_truck_l',
    'hourly_sb_bus_r', 'hourly_sb_bus_t', 'hourly_sb_bus_l',
    'hourly_nb_bus_r', 'hourly_nb_bus_t', 'hourly_nb_bus_l',
    'hourly_wb_bus_r', 'hourly_wb_bus_t', 'hourly_wb_bus_l',
    'hourly_eb_bus_r', 'hourly_eb_bus_t', 'hourly_eb_bus_l',
    'hourly_nx_peds', 'hourly_sx_peds', 'hourly_ex_peds', 'hourly_wx_peds',
    'hourly_nx_bike', 'hourly_sx_bike', 'hourly_ex_bike', 'hourly_wx_bike',
    'hourly_nx_other', 'hourly_sx_other', 'hourly_ex_other', 'hourly_wx_other'
    ]

    # Sum the columns along axis=1 to get the total for each row
    data['total_daily_traffic'] = data[daily_traffic].sum(axis=1)
    data['total_hourly_traffic'] = data[hourly_traffic].sum(axis=1)
    
    #groupby neighbourhood and get the average traffic for each neighbourhood
    average_traffic_per_neighborhood = data.groupby('NEIGHBOURHOOD_158')[['total_daily_traffic', 'total_hourly_traffic']].mean().reset_index()

    # Merge the average_traffic_per_neighborhood DataFrame back into the original data DataFrame
    data = data.merge(average_traffic_per_neighborhood, on='NEIGHBOURHOOD_158', how='left', suffixes=('', '_avg_neighborhood'))
    
    
    #FOUR
    #groupby ward and get the average traffic for each ward
    average_traffic_per_ward = data.groupby('WARDNUM')[['total_daily_traffic', 'total_hourly_traffic']].mean().reset_index()

    # Merge the average_traffic_per_neighborhood DataFrame back into the original data DataFrame
    data = data.merge(average_traffic_per_ward, on='WARDNUM', how='left', suffixes=('', '_avg_ward'))
    
    # MODIFIED FROM ORIGINAL CODE COPIED FROM GITHUB TO ALSO INCLUDE 'r' COLUMN AND STREET LOCATION
    feature_selected = ['count_location','collision_ratio_neighbourhood', 'collision_ratio_ward',
                        'total_daily_traffic_avg_neighborhood', 'total_hourly_traffic_avg_neighborhood', #neighbourhood traffic avgs
                       'total_daily_traffic_avg_ward', 'total_hourly_traffic_avg_ward', 'r'
                       ]
        
    return data[feature_selected]

# Chapter 1: Features based on neighbourhood and ward district

## 2.1.1 Import Datafiles

In [3]:
master_att_gdf = pd.read_csv('master_att_gdf.csv')
master_att_gdf.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates
0,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
1,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
2,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
3,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
4,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"


In [4]:
intersection_r = pd.read_csv('intersection_r.csv')
intersection_r.head()

,count_location,ROAD_CLASS,TRAFFCTL,num_of_collisions,total_traffic,average_int_width,years,r
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,18,20083.944444,0.015,11,1.488150
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,3,11620.000000,0.015,11,0.428685
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,10,9440.000000,0.015,11,1.758941
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,2,11585.000000,0.015,11,0.286653
4,KING ST E AT PRINCESS ST,Major Arterial,No Control,11,5304.000000,0.015,11,3.443597


## 2.1.2 Feature engineer master df and merge datasets

In [5]:
# Merge r dataset with master prior to feature engineering (note, function modified from original GH version)
result_df = pd.merge(master_att_gdf, intersection_r[['count_location', 'r']], on='count_location', how='left')
result_df.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates,r
0,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]",3.443597
1,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]",3.443597
2,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]",3.443597
3,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]",3.443597
4,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]",3.443597


In [6]:
# Feature engineering to produce features for feeding. Note that prior to model usage, count_location must be extracted.
feat_master_df = feature_engineering(result_df)
feat_master_df.head()

,count_location,collision_ratio_neighbourhood,collision_ratio_ward,total_daily_traffic_avg_neighborhood,total_hourly_traffic_avg_neighborhood,total_daily_traffic_avg_ward,total_hourly_traffic_avg_ward,r
0,KING ST E AT PRINCESS ST,0.020444,0.035668,13882.042553,1832.829787,15073.329268,1749.585366,3.443597
1,KING ST E AT PRINCESS ST,0.020444,0.035668,13882.042553,1832.829787,15073.329268,1749.585366,3.443597
2,KING ST E AT PRINCESS ST,0.020444,0.035668,13882.042553,1832.829787,15073.329268,1749.585366,3.443597
3,KING ST E AT PRINCESS ST,0.020444,0.035668,13882.042553,1832.829787,15073.329268,1749.585366,3.443597
4,KING ST E AT PRINCESS ST,0.020444,0.035668,13882.042553,1832.829787,15073.329268,1749.585366,3.443597


## 2.2.1 Train Test Split

In [7]:
# Perform the dataset split. Train will be used alongside KFold cross-validation.
X = feat_master_df.drop(columns = 'r')
y = feat_master_df['r']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2069 entries, 1399 to 1653
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   count_location                         2069 non-null   object 
 1   collision_ratio_neighbourhood          2069 non-null   float64
 2   collision_ratio_ward                   2069 non-null   float64
 3   total_daily_traffic_avg_neighborhood   2069 non-null   float64
 4   total_hourly_traffic_avg_neighborhood  2069 non-null   float64
 5   total_daily_traffic_avg_ward           2069 non-null   float64
 6   total_hourly_traffic_avg_ward          2069 non-null   float64
dtypes: float64(6), object(1)
memory usage: 129.3+ KB


In [9]:
len(y_train)

2069

In [10]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 648 to 1485
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   count_location                         230 non-null    object 
 1   collision_ratio_neighbourhood          230 non-null    float64
 2   collision_ratio_ward                   230 non-null    float64
 3   total_daily_traffic_avg_neighborhood   230 non-null    float64
 4   total_hourly_traffic_avg_neighborhood  230 non-null    float64
 5   total_daily_traffic_avg_ward           230 non-null    float64
 6   total_hourly_traffic_avg_ward          230 non-null    float64
dtypes: float64(6), object(1)
memory usage: 14.4+ KB


In [11]:
len(y_test)

230

## 2.3.1 Initial Model CV run with all features, with minor hyperparameter tuning

This section quickly explores the performance of an XGB model wiht no hyperparameter tuning. CV is used as a means to see how the MAE score can vary by using different sets of the data, prior to using the train vs test dataset.

In [12]:
run = 1

kf = KFold(n_splits = 5)

overall_score = []

X_cv1 = X_train.drop(columns = 'count_location')
    
y_cv1 = y_train

for train_rows, val_rows in kf.split(X_cv1, y_cv1):   
            
    X_cv1_feat_train, X_cv1_feat_val = (X_cv1.iloc[train_rows, :], 
                          X_cv1.iloc[val_rows, :]
                         )
    
    y_cv1_feat_train, y_cv1_feat_val = (y_cv1.iloc[train_rows], 
                          y_cv1.iloc[val_rows]
                          )
            
    test_model = XGBRegressor(random_state = 0)
    
    test_model.fit(X_cv1_feat_train, y_cv1_feat_train)
    
    feat_val_pred = test_model.predict(X_cv1_feat_val)
    feat_train_pred = test_model.predict(X_cv1_feat_train)
    
    err_test_val = mae(y_cv1_feat_val, feat_val_pred)
    err_train_val = mae(y_cv1_feat_train, feat_train_pred)
    
    overall_score.append(err_test_val)
        
    print('Val MAE is {}, and train MAE is {} on run {}.'.format(err_test_val, err_train_val, run))
    
    run += 1
    
print('Average validation MAE is {}.' .format(np.mean(overall_score)))

Val MAE is 0.42291645313726306, and train MAE is 0.3616346908653897 on run 1.
Val MAE is 0.4432809819582548, and train MAE is 0.3575669834421174 on run 2.
Val MAE is 0.4396040164669148, and train MAE is 0.3582724457577956 on run 3.
Val MAE is 0.39076956865943535, and train MAE is 0.3765141747776064 on run 4.
Val MAE is 0.402818793757489, and train MAE is 0.372887151725424 on run 5.
Average validation MAE is 0.4198779627958714.


Relatively minor variability in the CV implies that the features and model is relatively stable.

In [13]:
feature_importances = test_model.feature_importances_
column_names = X_train.drop(columns = 'count_location').columns.to_numpy()
feature_importance = pd.DataFrame(column_names)
feature_importance['importance'] = feature_importances
feature_importance = feature_importance.sort_values(by = ['importance'], ascending = False)
feature_importance.head(30)

,0,importance
3,total_hourly_traffic_avg_neighborhood,0.334279
2,total_daily_traffic_avg_neighborhood,0.204824
0,collision_ratio_neighbourhood,0.146784
1,collision_ratio_ward,0.123467
5,total_hourly_traffic_avg_ward,0.108683
4,total_daily_traffic_avg_ward,0.081963


## 2.4.1 Train vs Test
Final assessment of model with full training set against train.

In [14]:
final_model = XGBRegressor(random_state = 0)

final_model.fit(X_train.drop(columns = 'count_location'), y_train)

predictions = final_model.predict(X_test.drop(columns = 'count_location'))

print('Test score with full train dataset is {}.' .format(mae(y_test, predictions)))

Test score with full train dataset is 0.4769044949052575.


# Chapter 2: Features based on Intersection Attributes

## 3.1.1 Import files and feature engineer (reimport them to avoid conflicts)
Note: for this section, this is based off of seciton 3 of the Intersection Characteristics Analysis of master_gdf.ipynb notebook on GitHub.

In [15]:
# Read the CSV file into a DataFrame
master_att_gdf = pd.read_csv('master_att_gdf.csv')
master_att_gdf

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates
0,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
1,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
2,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
3,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
4,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,NaN,0.0,NaN,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2294,2002494669,2022.0,2022-12-21,1750,2022-12-21 17:50:00,GLOUCESTER GRV,WINNETT AVE,None,Local,None,...,0.0,NaN,0.0,NaN,8,200000,-1,5405,MultiPoint,"[[-79.4359754197162, 43.6955080379418]]"
2295,2002494669,2022.0,2022-12-21,1750,2022-12-21 17:50:00,GLOUCESTER GRV,WINNETT AVE,None,Local,None,...,0.0,NaN,0.0,NaN,8,200000,-1,5405,MultiPoint,"[[-79.4359754197162, 43.6955080379418]]"
2296,2002494669,2022.0,2022-12-21,1750,2022-12-21 17:50:00,GLOUCESTER GRV,WINNETT AVE,None,Local,None,...,0.0,NaN,0.0,NaN,8,200000,-1,5405,MultiPoint,"[[-79.4359754197162, 43.6955080379418]]"
2297,2002494669,2022.0,2022-12-21,1750,2022-12-21 17:50:00,GLOUCESTER GRV,WINNETT AVE,None,Local,None,...,0.0,NaN,0.0,NaN,8,200000,-1,5405,MultiPoint,"[[-79.4359754197162, 43.6955080379418]]"


In [16]:
# Read the CSV file into a DataFrame
master_intersection_gdf = pd.read_csv('intersection_r.csv')
master_intersection_gdf

,count_location,ROAD_CLASS,TRAFFCTL,num_of_collisions,total_traffic,average_int_width,years,r
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,18,20083.944444,0.015,11,1.488150
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,3,11620.000000,0.015,11,0.428685
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,10,9440.000000,0.015,11,1.758941
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,2,11585.000000,0.015,11,0.286653
4,KING ST E AT PRINCESS ST,Major Arterial,No Control,11,5304.000000,0.015,11,3.443597
...,...,...,...,...,...,...,...,...
1696,KINGSTON RD AT MARKHAM RD,Major Arterial,Traffic Signal,2,20556.000000,0.015,11,0.161553
1697,ALBION RD AT BENSTROW AVE & SANAGAN RD (PX 1953),Major Arterial,No Control,2,14765.000000,0.015,11,0.224916
1698,AVENUE RD AT LONSDALE RD (PX 113),Major Arterial,Traffic Signal,2,19238.000000,0.015,11,0.172621
1699,AVA RD AT WINNETT AVE,Local,Stop Sign,5,2427.000000,0.015,11,3.420766


### Including Elevations Columns to master_intersection_gdf

In [17]:
# Merge the dataframes based on 'count_location'
merged_df = pd.merge(master_intersection_gdf, master_att_gdf[['count_location', 'NUMBER_OF_ELEVATIONS']], on='count_location', how='left')

# Find the first instance of each intersection
first_instance_df = merged_df.groupby('count_location')['NUMBER_OF_ELEVATIONS'].first().reset_index()

# Rename the columns for clarity
first_instance_df.columns = ['count_location', 'number_of_elevations']

# Merge the new column back into master_intersection_gdf
master_intersection_gdf = pd.merge(master_intersection_gdf, first_instance_df, on='count_location', how='left')

# Fill NaN values with a default value if needed
master_intersection_gdf['number_of_elevations'].fillna(0, inplace=True)

In [18]:
master_intersection_gdf['number_of_elevations']

0       0.0
1       0.0
2       0.0
3       0.0
4       1.0
       ... 
1696    1.0
1697    0.0
1698    0.0
1699    1.0
1700    0.0
Name: number_of_elevations, Length: 1701, dtype: float64

### Identifying Road Classes High Historical Collisions

In [19]:
def add_major_arterial_column(df):
    # Create a new column 'major_arterial' with values 1 for 'Major Arterial', 0 otherwise
    df['major_arterial'] = (df['ROAD_CLASS'] == 'Major Arterial').astype(int)
    
    return df

In [20]:
#Checking if this works
master_intersection_gdf = add_major_arterial_column(master_intersection_gdf)
master_intersection_gdf

,count_location,ROAD_CLASS,TRAFFCTL,num_of_collisions,total_traffic,average_int_width,years,r,number_of_elevations,major_arterial
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,18,20083.944444,0.015,11,1.488150,0.0,1
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,3,11620.000000,0.015,11,0.428685,0.0,1
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,10,9440.000000,0.015,11,1.758941,0.0,0
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,2,11585.000000,0.015,11,0.286653,0.0,0
4,KING ST E AT PRINCESS ST,Major Arterial,No Control,11,5304.000000,0.015,11,3.443597,1.0,1
...,...,...,...,...,...,...,...,...,...,...
1696,KINGSTON RD AT MARKHAM RD,Major Arterial,Traffic Signal,2,20556.000000,0.015,11,0.161553,1.0,1
1697,ALBION RD AT BENSTROW AVE & SANAGAN RD (PX 1953),Major Arterial,No Control,2,14765.000000,0.015,11,0.224916,0.0,1
1698,AVENUE RD AT LONSDALE RD (PX 113),Major Arterial,Traffic Signal,2,19238.000000,0.015,11,0.172621,0.0,1
1699,AVA RD AT WINNETT AVE,Local,Stop Sign,5,2427.000000,0.015,11,3.420766,1.0,0


### Assigning Fatality Ratios to each Form of Traffic Control

In [21]:
def calculate_fatality_ratios(master_att_gdf):
    # Filter out rows with 'None' in ACCLASS
    filtered_data = master_att_gdf[master_att_gdf['ACCLASS'] != 'None']
    
    # Calculate fatality ratios for each TRAFFCTL category
    fatality_ratios_ctl = {}
    for traffctl_category in filtered_data['TRAFFCTL'].unique():
        subset = filtered_data[filtered_data['TRAFFCTL'] == traffctl_category]
        fatal_count = subset[subset['ACCLASS'] == 'Fatal'].shape[0]
        non_fatal_count = subset[subset['ACCLASS'] == 'Non-Fatal Injury'].shape[0]
        property_damage_count = subset[subset['ACCLASS'] == 'Property Damage Only'].shape[0]
        total_count = fatal_count + non_fatal_count + property_damage_count
        fatality_ratio = fatal_count / total_count if total_count > 0 else 0
        fatality_ratios_ctl[traffctl_category] = fatality_ratio
    
    return fatality_ratios_ctl

def add_fatality_ratio_ctl_column(df, fatality_ratios_ctl):
    # Create a new column 'fatality_ratio_ctl' based on 'TRAFFCTL'
    df['fatality_ratio_ctl'] = df['TRAFFCTL'].map(fatality_ratios_ctl)
    
    return df

In [22]:
#Checking if this works
# Calculate fatality ratios
fatality_ratios_ctl = calculate_fatality_ratios(master_att_gdf)

# Add 'fatality_ratio_ctl' column to master_intersection_gdf
master_intersection_gdf = add_fatality_ratio_ctl_column(master_intersection_gdf, fatality_ratios_ctl)
master_intersection_gdf

,count_location,ROAD_CLASS,TRAFFCTL,num_of_collisions,total_traffic,average_int_width,years,r,number_of_elevations,major_arterial,fatality_ratio_ctl
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,18,20083.944444,0.015,11,1.488150,0.0,1,0.188329
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,3,11620.000000,0.015,11,0.428685,0.0,1,0.130939
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,10,9440.000000,0.015,11,1.758941,0.0,0,0.130939
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,2,11585.000000,0.015,11,0.286653,0.0,0,0.188329
4,KING ST E AT PRINCESS ST,Major Arterial,No Control,11,5304.000000,0.015,11,3.443597,1.0,1,0.188329
...,...,...,...,...,...,...,...,...,...,...,...
1696,KINGSTON RD AT MARKHAM RD,Major Arterial,Traffic Signal,2,20556.000000,0.015,11,0.161553,1.0,1,0.130939
1697,ALBION RD AT BENSTROW AVE & SANAGAN RD (PX 1953),Major Arterial,No Control,2,14765.000000,0.015,11,0.224916,0.0,1,0.188329
1698,AVENUE RD AT LONSDALE RD (PX 113),Major Arterial,Traffic Signal,2,19238.000000,0.015,11,0.172621,0.0,1,0.130939
1699,AVA RD AT WINNETT AVE,Local,Stop Sign,5,2427.000000,0.015,11,3.420766,1.0,0,0.157480


### Identifying Intersection on Uneven Terrain

In [23]:
def add_uneven_terrain_column(df):
    # Create a new column 'uneven_terrain'
    df['uneven_terrain'] = df['number_of_elevations'].apply(lambda x: 1 if x > 1 else 0)
    return df

In [24]:
#Checking if this works
master_intersection_gdf = add_uneven_terrain_column(master_intersection_gdf)
master_intersection_gdf.head()

,count_location,ROAD_CLASS,TRAFFCTL,num_of_collisions,total_traffic,average_int_width,years,r,number_of_elevations,major_arterial,fatality_ratio_ctl,uneven_terrain
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,18,20083.944444,0.015,11,1.488150,0.0,1,0.188329,0
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,3,11620.000000,0.015,11,0.428685,0.0,1,0.130939,0
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,10,9440.000000,0.015,11,1.758941,0.0,0,0.130939,0
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,2,11585.000000,0.015,11,0.286653,0.0,0,0.188329,0
4,KING ST E AT PRINCESS ST,Major Arterial,No Control,11,5304.000000,0.015,11,3.443597,1.0,1,0.188329,0


In [25]:
# final datset
master_intersection_gdf = master_intersection_gdf.drop(columns = ['num_of_collisions', 
                                                                  'total_traffic',
                                                                  'average_int_width',
                                                                  'years']
                                                      )
master_intersection_gdf.head()

,count_location,ROAD_CLASS,TRAFFCTL,r,number_of_elevations,major_arterial,fatality_ratio_ctl,uneven_terrain
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,1.488150,0.0,1,0.188329,0
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,0.428685,0.0,1,0.130939,0
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,1.758941,0.0,0,0.130939,0
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,0.286653,0.0,0,0.188329,0
4,KING ST E AT PRINCESS ST,Major Arterial,No Control,3.443597,1.0,1,0.188329,0


## 3.2.1 Dummy encoding Categorical labels (Road Class)
Since we have 2 features that are categorical labels, we will need to use encoding in order for them to become useful for the model We will also have to set this up as functions to apply dnamically to datasets.

In [26]:
train_copy = master_intersection_gdf.copy()
# First, find what kind of labels are available for Road Class
train_copy['ROAD_CLASS'].unique()

array(['Major Arterial', 'Minor Arterial', 'Local', 'None', 'Collector',
       'Other', 'Expressway', 'Major Arterial Ramp'], dtype=object)

In [27]:
# Dummy encode for Road Class
def roadclass_enc(data):
    """
    One-hot-encodes Road classes
    """
    # List categories
    categories = ['Minor Arterial', 'Major Arterial', 'Collector', 'Local', 'None',
       'Major Arterial Ramp', 'Expressway', 'Other']

    category_type = pd.CategoricalDtype(categories=categories)
    data.loc[:, 'ROAD_CLASS'] = data.loc[:, 'ROAD_CLASS'].astype(category_type)
    data = pd.get_dummies(data, 
                          prefix='rc',
                          columns=['ROAD_CLASS'],
                          # Had to modify code due to different version issues. Code will work as originally intended
                          dtype = int).drop(columns = ['rc_Other'])
    
    return data

# Encode 'Fireplace_Qu'
rc_result = roadclass_enc(train_copy)
rc_result = rc_result

# View new encoded features
rc_result.head()

,count_location,TRAFFCTL,r,number_of_elevations,major_arterial,fatality_ratio_ctl,uneven_terrain,rc_Minor Arterial,rc_Major Arterial,rc_Collector,rc_Local,rc_None,rc_Major Arterial Ramp,rc_Expressway
0,BATHURST ST AT COLLEGE ST (PX 300),No Control,1.488150,0.0,1,0.188329,0,0,1,0,0,0,0,0
1,JANE ST AT ST JOHNS RD (PX 523),Traffic Signal,0.428685,0.0,1,0.130939,0,0,1,0,0,0,0,0
2,CHURCH ST AT GOULD ST (PX 993),Traffic Signal,1.758941,0.0,0,0.130939,0,1,0,0,0,0,0,0
3,CHARLES ST E AT CHURCH ST (PX 225),No Control,0.286653,0.0,0,0.188329,0,1,0,0,0,0,0,0
4,KING ST E AT PRINCESS ST,No Control,3.443597,1.0,1,0.188329,0,0,1,0,0,0,0,0


## 3.2.2 Dummy encoding Categorical labels (Traffic Control)

In [28]:
train_copy['TRAFFCTL'].unique()

array(['No Control', 'Traffic Signal', 'Stop Sign',
       'Pedestrian Crossover', 'Streetcar (Stop for)',
       'Traffic Controller', 'Yield Sign'], dtype=object)

In [29]:
# Dummy encode for Road Class
def trafctl_enc(data):
    """
    One-hot-encodes Traffic Control
    """
    # List categories
    categories = ['No Control', 'Stop Sign', 'Traffic Signal',
       'Pedestrian Crossover', 'Streetcar (Stop for)',
       'Traffic Controller', 'Yield Sign']

    category_type = pd.CategoricalDtype(categories=categories)
    data.loc[:, 'TRAFFCTL'] = data.loc[:, 'TRAFFCTL'].astype(category_type)
    data = pd.get_dummies(data, 
                          prefix='tfcl',
                          columns=['TRAFFCTL'],
                          # Had to modify code due to different version issues. Code will work as originally intended
                          dtype = int).drop(columns = ['tfcl_No Control'])
    
    return data

# Encode 'Fireplace_Qu'
tf_result = trafctl_enc(train_copy)
tf_result = tf_result

# View new encoded features
tf_result.head()

,count_location,ROAD_CLASS,r,number_of_elevations,major_arterial,fatality_ratio_ctl,uneven_terrain,tfcl_Stop Sign,tfcl_Traffic Signal,tfcl_Pedestrian Crossover,tfcl_Streetcar (Stop for),tfcl_Traffic Controller,tfcl_Yield Sign
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,1.488150,0.0,1,0.188329,0,0,0,0,0,0,0
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,0.428685,0.0,1,0.130939,0,0,1,0,0,0,0
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,1.758941,0.0,0,0.130939,0,0,1,0,0,0,0
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,0.286653,0.0,0,0.188329,0,0,0,0,0,0,0
4,KING ST E AT PRINCESS ST,Major Arterial,3.443597,1.0,1,0.188329,0,0,0,0,0,0,0


## 3.3.1 Train and Test Split

In [30]:
# Perform the dataset split. Train will be used alongside KFold cross-validation.
master_intersection_gdf = roadclass_enc(master_intersection_gdf)
master_intersection_gdf = trafctl_enc(master_intersection_gdf)

X = master_intersection_gdf.drop(columns = 'r')
y = master_intersection_gdf['r']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [31]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1530 entries, 674 to 684
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   count_location             1530 non-null   object 
 1   number_of_elevations       1530 non-null   float64
 2   major_arterial             1530 non-null   int32  
 3   fatality_ratio_ctl         1529 non-null   float64
 4   uneven_terrain             1530 non-null   int64  
 5   rc_Minor Arterial          1530 non-null   int32  
 6   rc_Major Arterial          1530 non-null   int32  
 7   rc_Collector               1530 non-null   int32  
 8   rc_Local                   1530 non-null   int32  
 9   rc_None                    1530 non-null   int32  
 10  rc_Major Arterial Ramp     1530 non-null   int32  
 11  rc_Expressway              1530 non-null   int32  
 12  tfcl_Stop Sign             1530 non-null   int32  
 13  tfcl_Traffic Signal        1530 non-null   int3

In [32]:
len(y_train)

1530

In [33]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 506 to 1223
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   count_location             171 non-null    object 
 1   number_of_elevations       171 non-null    float64
 2   major_arterial             171 non-null    int32  
 3   fatality_ratio_ctl         171 non-null    float64
 4   uneven_terrain             171 non-null    int64  
 5   rc_Minor Arterial          171 non-null    int32  
 6   rc_Major Arterial          171 non-null    int32  
 7   rc_Collector               171 non-null    int32  
 8   rc_Local                   171 non-null    int32  
 9   rc_None                    171 non-null    int32  
 10  rc_Major Arterial Ramp     171 non-null    int32  
 11  rc_Expressway              171 non-null    int32  
 12  tfcl_Stop Sign             171 non-null    int32  
 13  tfcl_Traffic Signal        171 non-null    int3

In [34]:
len(y_test)

171

## 3.4.1 Initial Model CV run with all features, with minor hyperparameter tuning

This section quickly explores the performance of an XGB model wiht no hyperparameter tuning. CV is used as a means to see how the MAE score can vary by using different sets of the data, prior to using the train vs test dataset.

In [35]:
X_cv1 = X_train.drop(columns = 'count_location')
    
y_cv1 = y_train

X_cv1.head()

,number_of_elevations,major_arterial,fatality_ratio_ctl,uneven_terrain,rc_Minor Arterial,rc_Major Arterial,rc_Collector,rc_Local,rc_None,rc_Major Arterial Ramp,rc_Expressway,tfcl_Stop Sign,tfcl_Traffic Signal,tfcl_Pedestrian Crossover,tfcl_Streetcar (Stop for),tfcl_Traffic Controller,tfcl_Yield Sign
674,0.0,0,0.188329,0,1,0,0,0,0,0,0,0,0,0,0,0,0
593,1.0,0,0.157480,0,1,0,0,0,0,0,0,1,0,0,0,0,0
66,0.0,1,0.188329,0,0,1,0,0,0,0,0,0,0,0,0,0,0
85,0.0,0,0.188329,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1081,1.0,1,0.130939,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [36]:
run = 1

kf = KFold(n_splits = 5)

overall_score = []

for train_rows, val_rows in kf.split(X_cv1, y_cv1):   
            
    X_cv1_feat_train, X_cv1_feat_val = (X_cv1.iloc[train_rows, :], 
                          X_cv1.iloc[val_rows, :]
                         )
    
    y_cv1_feat_train, y_cv1_feat_val = (y_cv1.iloc[train_rows], 
                          y_cv1.iloc[val_rows]
                          )
            
    test_model = XGBRegressor(random_state = 0)
    
    test_model.fit(X_cv1_feat_train, y_cv1_feat_train)
    
    feat_val_pred = test_model.predict(X_cv1_feat_val)
    feat_train_pred = test_model.predict(X_cv1_feat_train)
    
    err_test_val = mae(y_cv1_feat_val, feat_val_pred)
    err_train_val = mae(y_cv1_feat_train, feat_train_pred)
    
    overall_score.append(err_test_val)
        
    print('Val MAE is {}, and train MAE is {} on run {}.'.format(err_test_val, err_train_val, run))
    
    run += 1
    
print('Average validation MAE is {}.' .format(np.mean(overall_score)))

Val MAE is 0.5138390046641536, and train MAE is 0.38039062403143575 on run 1.
Val MAE is 0.5027647829643753, and train MAE is 0.40880083719363836 on run 2.
Val MAE is 0.3838423741391693, and train MAE is 0.4298649775692637 on run 3.
Val MAE is 0.4044172291347327, and train MAE is 0.4270173642486233 on run 4.
Val MAE is 0.4474905440422531, and train MAE is 0.4195521666777255 on run 5.
Average validation MAE is 0.45047078698893683.


Relatively minor variability in the CV implies that the features and model is relatively stable.

In [37]:
feature_importances = test_model.feature_importances_
column_names = X_cv1.columns.to_numpy()
feature_importance = pd.DataFrame(column_names)
feature_importance['importance'] = feature_importances
feature_importance = feature_importance.sort_values(by = ['importance'], ascending = False)
feature_importance.head(30)

,0,importance
7,rc_Local,0.439390
11,tfcl_Stop Sign,0.281871
6,rc_Collector,0.163966
0,number_of_elevations,0.021910
12,tfcl_Traffic Signal,0.021775
9,rc_Major Arterial Ramp,0.019503
2,fatality_ratio_ctl,0.016792
1,major_arterial,0.011128
13,tfcl_Pedestrian Crossover,0.008840
15,tfcl_Traffic Controller,0.006150


## 3.5.1 Train vs Test
Final assessment of model with full training set against train.

In [38]:
final_model = XGBRegressor(random_state = 0)

X_final_train = X_train.copy()
X_final_train = X_final_train.drop(columns = 'count_location')

final_model.fit(X_final_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=6, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
X_final_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1530 entries, 674 to 684
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   number_of_elevations       1530 non-null   float64
 1   major_arterial             1530 non-null   int32  
 2   fatality_ratio_ctl         1529 non-null   float64
 3   uneven_terrain             1530 non-null   int64  
 4   rc_Minor Arterial          1530 non-null   int32  
 5   rc_Major Arterial          1530 non-null   int32  
 6   rc_Collector               1530 non-null   int32  
 7   rc_Local                   1530 non-null   int32  
 8   rc_None                    1530 non-null   int32  
 9   rc_Major Arterial Ramp     1530 non-null   int32  
 10  rc_Expressway              1530 non-null   int32  
 11  tfcl_Stop Sign             1530 non-null   int32  
 12  tfcl_Traffic Signal        1530 non-null   int32  
 13  tfcl_Pedestrian Crossover  1530 non-null   int3

In [40]:
X_final_test = X_test.copy()
X_final_test = X_final_test.drop(columns = 'count_location')
X_final_test.head()

,number_of_elevations,major_arterial,fatality_ratio_ctl,uneven_terrain,rc_Minor Arterial,rc_Major Arterial,rc_Collector,rc_Local,rc_None,rc_Major Arterial Ramp,rc_Expressway,tfcl_Stop Sign,tfcl_Traffic Signal,tfcl_Pedestrian Crossover,tfcl_Streetcar (Stop for),tfcl_Traffic Controller,tfcl_Yield Sign
506,0.0,1,0.130939,0,0,1,0,0,0,0,0,0,1,0,0,0,0
6,0.0,1,0.130939,0,0,1,0,0,0,0,0,0,1,0,0,0,0
182,1.0,1,0.188329,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1230,0.0,1,0.188329,0,0,1,0,0,0,0,0,0,0,0,0,0,0
655,1.0,1,0.157480,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [41]:
predictions = final_model.predict(X_final_test)

print('Test score with full train dataset is {}.' .format(mae(y_test, predictions)))

Test score with full train dataset is 0.41179620348764545.


# Conclusion
It seems possible to train a model to predict on an intersection how safe it really is with decent accuracy. Based on features and their order of importances like total_traffic and road class, the feasibility is there.